In [1]:
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import networkx as nx
import numpy as np
from math import dist
import copy
import matplotlib.pyplot as plt
##### model
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN


import warnings
import collections
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import torch
from torch_geometric.utils.convert import from_networkx


%matplotlib inline



/Users/yanircohen/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# crate a list to add dataframes to
awsc_list = list()

# list of files
files_list = ['../Datasets/attack.json']

# Load event names
category_file = open("../Utils/event_category.json")
event_categories = json.load(category_file)
category_file.close()

encoder = OneHotEncoder(handle_unknown='ignore')
label_encoder = LabelEncoder()

In [3]:
for file in files_list:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        
        # Append the dataframes
        awsc_list.append(pd.DataFrame.from_records(data['Records']))

In [4]:
awsc = pd.concat(awsc_list).reset_index(drop=True)

In [5]:
awsc[(awsc['eventSource']=='cognito-idp.amazonaws.com')]

,userAgent,eventID,userIdentity,eventType,sourceIPAddress,eventName,eventSource,recipientAccountId,requestParameters,awsRegion,...,apiVersion,readOnly,resources,additionalEventData,sharedEventID,vpcEndpointId,managementEvent,eventCategory,sessionCredentialFromConsole,tlsDetails


In [6]:
users = pd.json_normalize(awsc['userIdentity']).merge(awsc[['eventTime','eventName','eventSource','awsRegion','errorCode']], left_index=True,right_index=True)

In [7]:
users['userName'] = users['userName'].fillna(users['arn']).fillna(users['invokedBy']) \
.fillna(users['invokedBy']).fillna(users['principalId']).fillna('accountId')

In [8]:
users = users[['eventTime','type','userName','eventName','eventSource','awsRegion','errorCode']]
users = users.rename(columns = {'eventTime':'timestamp','userName':'Identifier','eventSource':'TargetService','errorCode':'Error'})


In [9]:
users.head()


,timestamp,type,Identifier,eventName,TargetService,awsRegion,Error
0,2017-02-12T19:57:06Z,Root,arn:aws:iam::811596193553:root,ListBuckets,s3.amazonaws.com,us-east-1,NaN
1,2017-02-12T19:59:10Z,Root,arn:aws:iam::811596193553:root,GetAccountPasswordPolicy,iam.amazonaws.com,us-east-1,NoSuchEntityException
2,2017-02-12T19:59:10Z,Root,arn:aws:iam::811596193553:root,GetAccountSummary,iam.amazonaws.com,us-east-1,NaN
3,2017-02-12T19:59:10Z,Root,arn:aws:iam::811596193553:root,ListAccountAliases,iam.amazonaws.com,us-east-1,NaN
4,2017-02-12T19:59:10Z,Root,arn:aws:iam::811596193553:root,ListMFADevices,iam.amazonaws.com,us-east-1,NaN


In [10]:
users['timestamp'] = pd.to_datetime(users['timestamp']).map(pd.Timestamp.timestamp).map(int)
users = users.sort_values('timestamp',ascending=True,ignore_index=True)

In [11]:
users['Identifier'] = users['Identifier'].apply(lambda x : x if (len(x.split(":"))!=2) else x.split(":")[1])
users.nunique()

timestamp        58649
type                 5
Identifier          27
eventName          383
TargetService       49
awsRegion           15
Error               60
dtype: int64

In [12]:
users['identity-target'] = "source-"+users['Identifier'] + "|" + "target-"+ users['TargetService']

In [13]:
users.head()

,timestamp,type,Identifier,eventName,TargetService,awsRegion,Error,identity-target
0,1486929426,Root,arn:aws:iam::811596193553:root,ListBuckets,s3.amazonaws.com,us-east-1,NaN,source-arn:aws:iam::811596193553:root|target-s...
1,1486929550,Root,arn:aws:iam::811596193553:root,GetAccountPasswordPolicy,iam.amazonaws.com,us-east-1,NoSuchEntityException,source-arn:aws:iam::811596193553:root|target-i...
2,1486929550,Root,arn:aws:iam::811596193553:root,GetAccountSummary,iam.amazonaws.com,us-east-1,NaN,source-arn:aws:iam::811596193553:root|target-i...
3,1486929550,Root,arn:aws:iam::811596193553:root,ListAccountAliases,iam.amazonaws.com,us-east-1,NaN,source-arn:aws:iam::811596193553:root|target-i...
4,1486929550,Root,arn:aws:iam::811596193553:root,ListMFADevices,iam.amazonaws.com,us-east-1,NaN,source-arn:aws:iam::811596193553:root|target-i...


In [14]:
users['Identifier'] = users['Identifier'].apply(lambda x : f"source-{x}")
users['TargetService'] = users['TargetService'].apply(lambda x : f"target-{x}")

In [15]:
def get_time_df(start,end,minutes,df,col='timestamp'):
    minutes = minutes * 60
    df_list = []
    start_time = start
    while start < end:
        cur_df = df[(df[col] >= start_time) & (df[col] < start+minutes)]
        if len(cur_df):
            df_list.append(df[(df[col] >= start_time) & (df[col] < start+minutes)])
        start+=minutes
    return df_list

In [16]:
df_list = get_time_df(users['timestamp'].iloc[0],users['timestamp'].iloc[-1],60*60*24,users)

In [17]:
len(df_list)

38

# Create the graph

In [18]:
nodes = ['Identifier','TargetService']
edges = ['identity-target']


In [19]:
def add_nodes(g,df,col,x,jump=10):
    y = 10
    unique_nodes = df[col].unique()
    for i in unique_nodes:
        if i!= None:
            g.add_node(f'{i}',pos=(x,y))
            y+=jump
#     for i in range(len(unique_nodes)-1):
#         g.add_edge(unique_nodes[i],unique_nodes[i+1])

def add_edges(g,df,col):
    edges = df[col].value_counts()
    for node,value in edges.items():
        start_node,end_node = node.split('|')
        if "None" not in start_node and "None" not in end_node:
            g.add_edge(start_node, end_node, weight=1)

In [20]:
def generate_graph(G,df_nodes,df_edges):  
    x = 10
    nodes_li = []
    for node in nodes:
        add_nodes(G,df_nodes,node,x,100)
        x+=1
        nodes_li.append(df_nodes[node].unique())
    for i in range(len(nodes_li)-1):
        for node1 in nodes_li[i]:
            for node2 in nodes_li[i+1]:
                G.add_edge(node1,node2,weight=0)
    for edge in edges:
        add_edges(G,df_edges,edge)

In [21]:
G = nx.Graph()

In [22]:
generate_graph(G,users,users)

In [23]:
def generate_graphs(df,df_list):
    li = []
    for data in df_list:
        G = nx.Graph()
        generate_graph(G,df,data)
        li.append(G)
    return li

In [24]:
graphs = generate_graphs(users,df_list)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 14, 10
pos=nx.get_node_attributes(G,'pos')
# pos = nx.spring_layout(G, scale=20, k=3/np.sqrt(G.order()))
d = dict(G.degree)
nx.draw(G, pos, node_color='lightblue', 
        with_labels=True, 
        nodelist=d, 
        node_size=[d[k]*300 for k in d])
labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_size=5)
import matplotlib.pyplot as plt
plt.savefig(f'plotgraph.png', dpi=300, bbox_inches='tight')


pos=nx.get_node_attributes(G,'pos')
nx.draw(G,pos)
plt.show()

In [26]:
df = nx.to_pandas_adjacency(G, dtype=float)

In [27]:
df.head()

,source-arn:aws:iam::811596193553:root,source-backup,source-arn:aws:sts::811596193553:assumed-role/level6/level6-ec1335a148b902503664e2d162fcdaa98c224ea5,source-arn:aws:sts::811596193553:assumed-role/level6/level6,source-arn:aws:sts::811596193553:assumed-role/level5/level5,source-Level5,source-lambda.amazonaws.com,source-ec2.amazonaws.com,source-Level6,source-arn:aws:sts::811596193553:assumed-role/lambda_basic_execution/Level6,...,target-autoscaling.amazonaws.com,target-elasticfilesystem.amazonaws.com,target-codedeploy.amazonaws.com,target-kinesis.amazonaws.com,target-organizations.amazonaws.com,target-health.amazonaws.com,target-route53domains.amazonaws.com,target-storagegateway.amazonaws.com,target-events.amazonaws.com,target-ssm.amazonaws.com
source-arn:aws:iam::811596193553:root,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
source-backup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
source-arn:aws:sts::811596193553:assumed-role/level6/level6-ec1335a148b902503664e2d162fcdaa98c224ea5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
source-arn:aws:sts::811596193553:assumed-role/level6/level6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
source-arn:aws:sts::811596193553:assumed-role/level5/level5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.manifold import SpectralEmbedding
embedding = SpectralEmbedding(n_components=70)
X = embedding.fit_transform(df)

In [29]:
X

array([[ 0.06096073,  0.00046259, -0.00129354, ...,  0.01372656,
         0.0095844 , -0.03040142],
       [ 0.06068499,  0.00045591, -0.00124011, ..., -0.01026301,
        -0.00404375,  0.03309437],
       [ 0.05553023,  0.00037853, -0.00058072, ...,  0.02651808,
         0.02362312, -0.03481944],
       ...,
       [-0.04125625,  0.08401731, -0.03213018, ..., -0.02383787,
         0.00128078,  0.03358378],
       [-0.04125625,  0.08401731, -0.03213018, ..., -0.02383787,
         0.00128078,  0.03358378],
       [-0.04125625,  0.08401731, -0.03213018, ..., -0.02383787,
         0.00128078,  0.03358378]])

In [30]:
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])

for i, txt in enumerate(X):
    ax.annotate(i, (txt[0], txt[1]))

In [31]:
df_from_graphs = []
for graph in graphs:
    df_from_graphs.append(nx.to_pandas_adjacency(graph, dtype=float))

In [32]:
distance_df = df.replace(1,0).copy(deep=True)

In [45]:
distance_df

,source-arn:aws:iam::811596193553:root,source-backup,source-arn:aws:sts::811596193553:assumed-role/level6/level6-ec1335a148b902503664e2d162fcdaa98c224ea5,source-arn:aws:sts::811596193553:assumed-role/level6/level6,source-arn:aws:sts::811596193553:assumed-role/level5/level5,source-Level5,source-lambda.amazonaws.com,source-ec2.amazonaws.com,source-Level6,source-arn:aws:sts::811596193553:assumed-role/lambda_basic_execution/Level6,...,source-arn:aws:sts::811596193553:assumed-role/SecurityMonkey/secmonkey,source-arn:aws:sts::811596193553:assumed-role/SecurityMonkey/cloudaux,source-AIDAS6VXHZ3PBMIBVP1P2,source-config.amazonaws.com,source-arn:aws:sts::811596193553:assumed-role/config-role-us-west-2/AWSConfig-BucketConfigCheck,source-arn:aws:sts::811596193553:assumed-role/config-role-us-west-2/AWSConfig-Describe,source-AIDA8M82W7I80M2DIZCF5,source-arn:aws:sts::811596193553:assumed-role/SummitRouteAudit/audit,source-flaws,source-cloud_user
source-arn:aws:iam::811596193553:root,0.000000,4.738102,0.0,0.000000,0.000000,6.351655,0.000000,0.000000,7.317744,0.0,...,11.865440,0.0,0.0,0.0,0.0,11.545223,0.0,0.0,26.527569,0.0
source-backup,4.738102,0.000000,0.0,0.000000,14.333521,0.000000,0.000000,0.000000,5.128964,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-arn:aws:sts::811596193553:assumed-role/level6/level6-ec1335a148b902503664e2d162fcdaa98c224ea5,0.000000,0.000000,0.0,0.000000,4.542961,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-arn:aws:sts::811596193553:assumed-role/level6/level6,0.000000,0.000000,0.0,0.000000,4.259897,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-arn:aws:sts::811596193553:assumed-role/level5/level5,0.000000,0.000000,0.0,4.259897,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-Level5,6.351655,5.805076,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-lambda.amazonaws.com,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,3.963133,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-ec2.amazonaws.com,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.963133,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
source-Level6,7.317744,5.128964,0.0,0.000000,5.717828,6.652113,0.000000,0.000000,0.000000,0.0,...,12.493811,0.0,0.0,0.0,0.0,12.155677,0.0,0.0,27.663541,0.0
source-arn:aws:sts::811596193553:assumed-role/lambda_basic_execution/Level6,0.000000,0.000000,0.0,0.000000,4.542941,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [34]:
from sklearn.neighbors import NearestNeighbors
a = 0
for df_graph in df_from_graphs:
    a+=10
    target_cols = [col for col in df_graph.columns if 'target' in col]
    for col in target_cols:
        index_di = {}
        index_to_pd_index = {}
        nodes = df_graph[col].loc[df_graph[col]==1]
        for i,other in enumerate(nodes.items()):
            ind,_=other
            pd_index = df_graph.index.get_loc(ind)
            index_di[pd_index] = ind
            index_to_pd_index[i] = ind
        if len(index_di) > 1:
            x = X[list(index_di.keys())]
            nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(x)
            distances, indices = nbrs.kneighbors(x)
            for i,other in enumerate(index_di.items()):
                key,value = other
                nearest_neighbor = indices[i][1]
                nearest_neighbor_ind = index_to_pd_index[nearest_neighbor]
                if distance_df.loc[value,nearest_neighbor_ind] == 0:
                    distance_df.loc[value,nearest_neighbor_ind] = distances[i][1]*a

In [36]:
cols = [c for c in distance_df.columns if 'target' not in c]

distance_df = distance_df[cols].drop(distance_df.columns.difference(cols))

In [37]:
result = nx.from_pandas_adjacency(distance_df)

In [44]:
color_map = ['red' if node == 'source-cloud_user' else 'green' for node in result]        
# pos=nx.spring_layout(result) # pos = nx.nx_agraph.graphviz_layout(G)
nx.draw(result,node_color=color_map, with_labels = True)
# labels = nx.get_edge_attributes(result,'weight')
# nx.draw_networkx_edge_labels(result,pos=pos)
plt.savefig("res.png")